# # BERT + LoRA 및 기타 테크닉을 활용하여 Q/A 구현하기 (Large Version)


Author : 정상근 (hugmanskj@gmail.com)

> 알림>
> 교육용 목적으로 제작된 코드입니다.

이 노트북에서는 BERT 모델을 LoRa를 이용하여 메모리를 적게 사용하는 방법을 실습합니다.
활용 데이터는 Suqad이며, **BERT Large** + LoRA를 이용하여 Q/A문제를 구현해봅니다.

이번 실습에서는 좀 더 큰 BERT를 활용하기 때문에 GPU 메모리가 부족할 수 있습니다. 이런 문제를 해결하기 위해 Quantization 및 기타 테크과 함께 LoRA를 동시에 활용해보도록 하겠습니다.

아래의 소프트웨어와 프레임워크를 사용해서 진행하겠습니다.

(훈련 소요 시간 : 1시간 20분)

In [126]:
from IPython.display import Markdown, display

In [127]:
import os

# 첫 번째 GPU만 사용하도록 설정 (본 실습환경 때문)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["WANDB_MODE"] = "disabled"

## 향후 쓰일 여러가지 utility function 준비

In [128]:
from pynvml import *

def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params:,} || all params: {all_param:,} || trainable : {100 * trainable_params / all_param}%"
    )
    return trainable_params


def compare_param(ori_p, peft_p):
    """
    Compare two parameter numbers
    """
    print(f"\n# Trainable Parameter \nBefore: {ori_p:>14,d} \nAfter:  {peft_p:>14,d} \nPercentage: {round(peft_p / ori_p * 100, 2)}%")


def show_trainable_structure(model):
    """
    Print only the trainable parameters only with size
    """
    num_totals = 0
    for _, param in model.named_parameters():
        if param.requires_grad:
            print( f"{_} \t {param.shape} \t\t {param.numel():,}" )
            num_totals += 1

    print(f"\nTotal Number of Parameter Names : {num_totals:,}")

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()


## Squad Dataset

아래 사이트에서 Dataset을 살펴볼 수 있습니다.

https://huggingface.co/datasets/squad

### Dataset 준비

In [129]:
## It takes time! (about 10min)
from datasets import load_dataset
dataset = load_dataset("squad")

In [130]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

### Dataset 살펴보기

Squad dataset 은 Context가 주어지고, 이 Context로부터의 Question 과 Answer가 각각 주어집니다.

In [131]:
dataset["train"][0]

{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}

In [132]:
dataset["train"][0].keys()

dict_keys(['id', 'title', 'context', 'question', 'answers'])

In [133]:
print("Title : ", dataset["train"][0]['title'])
print("-"*50)
print("[Context]")
print(dataset["train"][0]['context'])
print("-"*50)
print("Question : ", dataset["train"][0]['question'])
print("Answer   : ", dataset["train"][0]['answers'])

Title :  University_of_Notre_Dame
--------------------------------------------------
[Context]
Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.
--------------------------------------------------
Question :  To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
Answer   :  {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]

위의 예제에서, 질문에 대한 답은 'Saint Bernadette Soubirous' 이며, 이 정답은 Context 의 515 Charcter 부터 존재한다는 것을 알 수 있습니다. 이를 확인해봅니다.

In [134]:
ans_len = len(dataset["train"][0]['answers']['text'][0])
start_pos = dataset["train"][0]['answers']['answer_start'][0]

## -- extract answer from the context
dataset["train"][0]['context'][ start_pos : start_pos + ans_len]

'Saint Bernadette Soubirous'

### Q/A dataset Preprocessing


Q/A Data를 처리하기 위해서 Tokenizatoin 등을 포함한 전처리를 진행해야 합니다.

#### Model ID 설정

In [135]:
MODEL_ID = "bert-large-uncased"

### Tokenization

In [136]:
import transformers
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

# fast version 인지 아닌지 확인
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

pad_on_right = tokenizer.padding_side == "right"
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

In [137]:
# Data to Feature fucntion
# from this link : https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/question_answering.ipynb#scrollTo=LP4YiUxrQ97Q
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [138]:
features = prepare_train_features(dataset['train'][:5])

In [139]:
features.keys()

KeysView({'input_ids': [[101, 2000, 3183, 2106, 1996, 6261, 2984, 9382, 3711, 1999, 8517, 1999, 10223, 26371, 2605, 1029, 102, 6549, 2135, 1010, 1996, 2082, 2038, 1037, 3234, 2839, 1012, 10234, 1996, 2364, 2311, 1005, 1055, 2751, 8514, 2003, 1037, 3585, 6231, 1997, 1996, 6261, 2984, 1012, 3202, 1999, 2392, 1997, 1996, 2364, 2311, 1998, 5307, 2009, 1010, 2003, 1037, 6967, 6231, 1997, 4828, 2007, 2608, 2039, 14995, 6924, 2007, 1996, 5722, 1000, 2310, 3490, 2618, 4748, 2033, 18168, 5267, 1000, 1012, 2279, 2000, 1996, 2364, 2311, 2003, 1996, 13546, 1997, 1996, 6730, 2540, 1012, 3202, 2369, 1996, 13546, 2003, 1996, 24665, 23052, 1010, 1037, 14042, 2173, 1997, 7083, 1998, 9185, 1012, 2009, 2003, 1037, 15059, 1997, 1996, 24665, 23052, 2012, 10223, 26371, 1010, 2605, 2073, 1996, 6261, 2984, 22353, 2135, 2596, 2000, 3002, 16595, 9648, 4674, 2061, 12083, 9711, 2271, 1999, 8517, 1012, 2012, 1996, 2203, 1997, 1996, 2364, 3298, 1006, 1998, 1999, 1037, 3622, 2240, 2008, 8539, 2083, 1017, 11342, 1998

In [140]:
# tokeized input ids
print( features['input_ids'][0] )

[101, 2000, 3183, 2106, 1996, 6261, 2984, 9382, 3711, 1999, 8517, 1999, 10223, 26371, 2605, 1029, 102, 6549, 2135, 1010, 1996, 2082, 2038, 1037, 3234, 2839, 1012, 10234, 1996, 2364, 2311, 1005, 1055, 2751, 8514, 2003, 1037, 3585, 6231, 1997, 1996, 6261, 2984, 1012, 3202, 1999, 2392, 1997, 1996, 2364, 2311, 1998, 5307, 2009, 1010, 2003, 1037, 6967, 6231, 1997, 4828, 2007, 2608, 2039, 14995, 6924, 2007, 1996, 5722, 1000, 2310, 3490, 2618, 4748, 2033, 18168, 5267, 1000, 1012, 2279, 2000, 1996, 2364, 2311, 2003, 1996, 13546, 1997, 1996, 6730, 2540, 1012, 3202, 2369, 1996, 13546, 2003, 1996, 24665, 23052, 1010, 1037, 14042, 2173, 1997, 7083, 1998, 9185, 1012, 2009, 2003, 1037, 15059, 1997, 1996, 24665, 23052, 2012, 10223, 26371, 1010, 2605, 2073, 1996, 6261, 2984, 22353, 2135, 2596, 2000, 3002, 16595, 9648, 4674, 2061, 12083, 9711, 2271, 1999, 8517, 1012, 2012, 1996, 2203, 1997, 1996, 2364, 3298, 1006, 1998, 1999, 1037, 3622, 2240, 2008, 8539, 2083, 1017, 11342, 1998, 1996, 2751, 8514, 1007

In [141]:
tokenizer.decode(features['input_ids'][0])

'[CLS] to whom did the virgin mary allegedly appear in 1858 in lourdes france? [SEP] architecturally, the school has a catholic character. atop the main building \' s gold dome is a golden statue of the virgin mary. immediately in front of the main building and facing it, is a copper statue of christ with arms upraised with the legend " venite ad me omnes ". next to the main building is the basilica of the sacred heart. immediately behind the basilica is the grotto, a marian place of prayer and reflection. it is a replica of the grotto at lourdes, france where the virgin mary reputedly appeared to saint bernadette soubirous in 1858. at the end of the main drive ( and in a direct line that connects through 3 statues and the gold dome ), is a simple, modern stone statue of mary. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [P

In [142]:
# tokeized token type ids
print( features['token_type_ids'][0] )

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

여기서 0값과 1값이 바뀌는 구간을 잘 확인해보세요

In [143]:
# attention mask
print( features['attention_mask'][0] )

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

[PAD] 파트 부터는 모두 mask 가 0 입니다.

In [144]:
# tokeized token type ids
print( "Start Pos : ", features['start_positions'][0] )
print( "End   Pos : ", features['end_positions'][0] )

Start Pos :  130
End   Pos :  137


In [145]:
start_p = features['start_positions'][0]
end_p = features['end_positions'][0]
tokenizer.decode( features['input_ids'][0][start_p:end_p+1] )

'saint bernadette soubirous'

tokenized_datasets 에 미리 dataset 전체를 tokenization 해 둡니다.

In [146]:
tokenized_datasets = dataset.map(
                        prepare_train_features,
                        batched=True,
                        remove_columns=dataset["train"].column_names
                    )

In [147]:
tokenized_datasets.keys()

dict_keys(['train', 'validation'])

In [148]:
print(dataset["train"].column_names)

['id', 'title', 'context', 'question', 'answers']


> 옵션 설명
1. batched=True 옵션은 datasets 라이브러리의 map 함수에서 매우 중요한 역할을 합니다. 이 옵션을 사용하면 map 함수가 데이터셋의 각 샘플을 개별적으로 처리하는 대신 배치 단위로 처리합니다.
2. remove_columns=datasets["train"].column_names 옵션은 datasets 라이브러리의 map 함수에서 사용되며, 특정 열(columns)을 결과 데이터셋에서 제거하는 데 사용됩니다.
3. 예를 들어, SQuAD 데이터셋의 경우 원본 열에는 '문맥(context)', '질문(question)', '답변(answer)' 등이 포함되어 있습니다. 토크나이징 과정을 거쳐 새로운 열(예: 'input_ids', 'attention_mask', 'start_positions', 'end_positions')이 추가되면, 원본의 '문맥', '질문', '답변' 열은 더 이상 필요하지 않을 수 있습니다.

테스트를 위해 작은 데이터 준비

In [149]:
# full data
#train_dataset = tokenized_datasets["train"]
#test_dataset  = tokenized_datasets["validation"]

# small data (for practice)
train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(7000))
test_dataset  = tokenized_datasets["validation"].shuffle(seed=42).select(range(1000))

이번 실습에서는, GPU 메모리에서 한번에 훈련하기 버거운 모델을 가져와서 몇가지 테크닉을 통해 훈련을 가능케 하는 테크닉을 살펴보도록 하겠습니다. 더불어 이러한 테크닉들은 LoRA랑도 같이 사용하실 수 있습니다.

## 기본 모델 살펴보기

In [150]:
## Model Loading
from transformers import AutoModelForQuestionAnswering

model = AutoModelForQuestionAnswering.from_pretrained(
    MODEL_ID)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [151]:
ori_p = print_trainable_parameters(model)

trainable params: 334,094,338 || all params: 334,094,338 || trainable : 100.0%


위 실험은 약 14119 MB 혹은 그 이상의 메모리를 사용할 것입니다.

BERT Large모델을 그대로 가져와서 쓰기 때문에 사용하는 GPU 의 메모리 용량에 따라 실행이 전혀 안되기도 할 것입니다. 예를 들어 Google Coab Free version 이거나 사용하는 GPU의 메모리가 15GB 정도라면, 실행이 중간에 멈출 것입니다.

다양한 테크닉을 써서 대단히 커다란 모델을 작은 GPU에 가져와서 쓸 수 있도록 해야 합니다.
이번 실습에서는 몇가지 테크닉을 소개하고, 이를 LoRA와 함께 적용하는 법을 살펴보겠습니다.

## Big Model 을 작은 GPU 에 넣는 기법들

실용적으로 사용할 만한 테크닉이 몇 가지를 소개합니다. 계속해서 새로운 테크닉이 나오고, 사용할 수 있는 툴도 다양해지고 있기 때문에 모든 것을 소개해드리기는 힘듭니다만, Huggingface 상황에서 쉽게 적용할 수 있는 몇가지 테크닉을 소개합니다.




### [모델 로딩시 적용] 8Bit 로 Model 로딩해오기
Pretrained Model 을 로딩해올 때, load_in_8bit=True 옵션은 딥러닝 모델 훈련과 추론 과정에서 메모리 사용량을 크게 줄이는 방법입니다. 이 옵션을 활성화하면 모델의 가중치와 활성화들이 8비트 정밀도로 저장되고 처리됩니다. 이 접근법의 주요 장점과 효과는 다음과 같습니다:

- 메모리 사용량 감소: 8비트 정밀도로 모델을 로드하면 기존 32비트(float) 형식에 비해 메모리 사용량이 대폭 감소합니다. 이는 특히 GPU 메모리가 제한된 상황에서 유용하며, 더 큰 모델을 사용하거나 더 큰 배치 크기로 훈련하는 데 도움이 됩니다.
- 훈련 및 추론 속도 향상: 메모리 사용량이 줄어들면 데이터 전송 속도가 빨라지고, 연산이 더 효율적으로 이루어질 수 있습니다. 결과적으로, 모델 훈련과 추론 속도가 향상될 수 있습니다.


#### 주의사항
- 정밀도와 성능: 8비트 정밀도는 32비트보다 낮기 때문에, 때때로 모델의 성능(정확도 등)에 영향을 줄 수 있습니다. 따라서 성능에 민감한 작업에서는 8비트와 32비트 모델의 성능을 비교하는 것이 중요합니다.
- 하드웨어 호환성: 모든 하드웨어가 8비트 연산을 지원하는 것은 아닙니다. 따라서 8비트 모델을 사용하기 전에 하드웨어의 호환성을 확인하는 것이 좋습니다.

----


### [모델에 적용] Gradient Checkpointing 적용하기
huggingface의 경우 'model.gradient_checkpointing_enable()' 라는 함수를 제공합니다. 이 함수는 딥러닝 모델, 특히 메모리 집약적인 대규모 모델의 훈련 시 메모리 효율성을 크게 향상시키는 기법인 Gradient Checkpointing을 활성화합니다.

표준 backpropagation에서 그래디언트를 계산하기 위해, 신경망은 forward pass 동안의 activations (각 레이어의 출력)을 저장합니다. 이 activations은 backward pass에서 그래디언트를 계산하는 데 사용됩니다. 하지만 많은 레이어를 가진 딥 네트워크에서 이 activations을 모두 저장하는 것은 상당한 양의 메모리를 소비할 수 있습니다.

이 기법의 주요 장점과 효과는 다음과 같습니다:

- 메모리 사용량 감소: Gradient Checkpointing은 전방향 전달(forward pass) 동안의 모든 활성화(activations)를 저장하는 대신, 특정 레이어에서만 활성화를 저장하고, 필요할 때 나머지를 다시 계산합니다. 이는 전체적인 메모리 사용량을 크게 줄여줍니다.

- 대규모 모델 훈련 가능: 이 기법을 사용하면 메모리 사용량이 감소함에 따라, 기존에는 GPU 메모리 제약으로 인해 훈련하기 어려웠던 대규모 모델도 훈련할 수 있게 됩니다.

- 배치 크기 증가: 메모리 절감으로 인해 더 큰 배치 크기로 훈련을 진행할 수 있게 되며, 이는 모델의 성능과 안정성에 긍정적인 영향을 미칠 수 있습니다.


#### 주의사항
- 계산 오버헤드: 활성화를 다시 계산해야 하므로, 훈련 시간이 다소 증가할 수 있습니다. 이는 메모리 절약과 계산 시간 사이의 트레이드오프를 고려해야 합니다.
- 하이퍼파라미터 조정: Gradient Checkpointing을 사용할 때는 학습률 및 다른 하이퍼파라미터를 조정할 필요가 있을 수 있습니다.

### [모델에 적용] LoRA 적용하기

LoRA(Low-Rank Adaptation)는 특히 사전 훈련된 딥러닝 모델, 예를 들어 Transformer 기반 모델의 효율적인 미세 조정을 가능하게 하는 기술입니다. lora_config를 사용하여 LoRA를 적용하면 다음과 같은 효과를 기대할 수 있습니다:

- 효율적인 파라미터 수정: LoRA는 모델의 기존 가중치를 직접 수정하지 않고, 저차원 행렬을 도입하여 기존 가중치에 적용합니다. 이 접근법은 모델의 파라미터 수를 크게 늘리지 않으면서도 사전 훈련된 모델의 성능을 개선할 수 있습니다.

- 빠른 미세 조정 및 적응: LoRA를 적용한 모델은 새로운 데이터 또는 특정 작업에 대해 빠르게 미세 조정될 수 있습니다. 이는 전체 모델을 처음부터 훈련시키는 것보다 시간과 자원을 절약할 수 있게 해줍니다.

- 유연한 훈련과 확장성: lora_config에서는 LoRA의 다양한 매개변수(예: 저차원 행렬의 랭크)를 조정할 수 있어, 모델의 크기와 복잡도를 효과적으로 관리할 수 있습니다. 이를 통해 모델의 용량과 성능 사이에서 균형을 맞출 수 있습니다.

- 기존 가중치 보존: LoRA는 사전 훈련된 모델의 가중치를 보존하면서 새로운 데이터에 대한 학습을 가능하게 합니다. 이는 전이 학습(Transfer Learning)에서 중요한 요소로, 모델이 기존에 학습한 지식을 유지하면서 새로운 정보를 통합할 수 있게 해줍니다.

##### 주의사항
- 하이퍼파라미터 조정: LoRA의 효과는 lora_config에서 설정한 하이퍼파라미터에 크게 의존합니다. 적절한 랭크와 학습률 등을 실험을 통해 찾는 것이 중요합니다.

- 특정 모델 및 작업에 최적화: LoRA는 특히 Transformer 기반 모델에 적합합니다. 따라서 모델의 종류와 특정 작업에 따라 LoRA의 적용 방식과 효과가 달라질 수 있습니다.

----

### [훈련 시 적용] fp16 옵션 켜기
fp16=True 설정은 딥러닝 모델 훈련 시 16비트 부동소수점(FP16, Half Precision)을 사용하도록 지정하는 옵션입니다. 이 설정은 모델 훈련의 메모리 사용량을 줄이고, 훈련 속도를 높이는 데 도움이 됩니다. fp16 모드의 주요 장점과 효과는 다음과 같습니다:

- 메모리 사용량 감소: FP16을 사용하면 32비트 부동소수점(FP32, Full Precision)에 비해 메모리 사용량이 절반으로 줄어듭니다. 이는 특히 GPU 메모리가 제한된 상황에서 유용하며, 더 큰 모델을 사용하거나 더 큰 배치 크기로 훈련하는 데 도움이 됩니다.

- 훈련 및 추론 속도 향상: 메모리 대역폭이 감소하고 연산 속도가 빨라지기 때문에, 전체적인 모델 훈련과 추론 속도가 향상됩니다. FP16은 특히 GPU에서의 행렬 연산에서 더 빠른 속도를 제공합니다.

#### 주의사항
- 정밀도와 성능: FP16은 FP32보다 낮은 정밀도를 가지므로, 때때로 모델의 성능(정확도 등)에 영향을 줄 수 있습니다. 성능 저하를 최소화하기 위해, 많은 딥러닝 프레임워크들은 FP16과 FP32를 혼합하여 사용하는 Mixed Precision 훈련을 지원합니다.

- 하드웨어 호환성: 모든 GPU가 FP16 연산을 최적화하여 지원하는 것은 아닙니다. 따라서 FP16 모드를 사용하기 전에 하드웨어의 호환성을 확인하는 것이 중요합니다.
----


### [훈련 시 적용] Graident Accumulation 사용하기

gradient_accumulation_steps 설정은 딥러닝 모델 훈련 시 배치에 대한 그래디언트를 즉시 업데이트하는 대신, 여러 스텝에 걸쳐 그래디언트를 누적(accumulate)한 후에 업데이트하는 방식을 정의합니다. 이 옵션의 주요 장점과 효과는 다음과 같습니다:

- 큰 배치 크기 효과: 실제 물리적 배치 크기는 제한되어 있지만, gradient_accumulation_steps를 증가시킴으로써 더 큰 배치 크기의 효과를 낼 수 있습니다. 이는 메모리 한계 때문에 큰 배치 크기를 직접 사용할 수 없는 경우에 특히 유용합니다.

- 메모리 효율성 증가: 더 큰 배치 크기를 사용하는 것과 유사한 효과를 내면서도, 각 스텝에서 처리하는 실제 데이터 양은 작게 유지되므로 GPU 메모리 사용량을 줄일 수 있습니다.

- 모델 성능 및 안정성 개선: 더 큰 배치 크기로 인한 이점을 통해 모델의 성능이 향상되고, 훈련 과정이 더 안정적으로 진행될 수 있습니다. 큰 배치 크기는 그래디언트 업데이트의 분산을 줄여주며, 이는 일반적으로 더 안정적인 최적화 과정으로 이어집니다.

#### 주의사항
- 훈련 시간: 그래디언트 누적은 실제 업데이트가 더 드물게 일어나므로, 같은 수의 에포크를 완료하는 데 더 긴 시간이 소요될 수 있습니다.

- 하이퍼파라미터 조정: gradient_accumulation_steps를 변경하면 학습률과 같은 다른 하이퍼파라미터를 재조정할 필요가 있을 수 있습니다.

## Big Model 을 작은 GPU 에 넣기

### 8Bit 로 Model 로딩해오기

In [152]:
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, TaskType

tokenizer_id = MODEL_ID

# 8bit quantization 설정
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.float32,
    bnb_8bit_use_double_quant=False,       # True에서 False로 변경
)

# 모델 로딩
model = AutoModelForQuestionAnswering.from_pretrained(
    MODEL_ID,
    quantization_config=quantization_config,
    device_map='auto',
    trust_remote_code=True,
    torch_dtype=torch.float32,  # 명시적으로 float32
)


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


> Note!

BERT-Large 수준은 충분히 최근 GPU에는 담을 수 있기 때문에 훈련 안정성에 영향을 줄 수 있는 각종 Quantization 테크닉을 굳이 사용할 필요는 없습니다. 특히 load_in_8bit=True, fp16=True 옵션등등은 상황에 따라 NaN값을 만들 수도 있습니다.
이번 실습에서는 교육용 목적을위해 성능과 상관없이 각 PEFT테크닉을 모두 적용해보도록 합니다.

In [153]:
ori_p = print_trainable_parameters(model)

trainable params: 31,883,266 || all params: 334,094,338 || trainable : 9.54319255778588%


### Gradient Checkpointing 적용하기

In [154]:
# Gradient Checkpointing 활성화
model.gradient_checkpointing_enable() # 훈련속도를 느리게 할 수 있습니다.

### LoRA 적용하기

In [155]:
from peft import LoraConfig, TaskType

lora_config = LoraConfig(
    task_type=TaskType.QUESTION_ANS,  # TASK TYPE
    r=16, # Rank
    lora_alpha=1,
    lora_dropout=0.1,
    #target_modules=["query", "value"],  # We can specify target Modules
)

from peft import get_peft_model
model = get_peft_model(model, lora_config)

In [156]:
lora_p = print_trainable_parameters(model)

trainable params: 1,574,914 || all params: 335,669,252 || trainable : 0.46918625719105184%


### FP16 및 Gradient Accumuation 적용하기

In [157]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    ## ---- Epoch & Batch ----- ##
    num_train_epochs=3,             # 훈련 에폭 수
    per_device_train_batch_size=4,  # 디바이스 당 훈련 배치 크기
    auto_find_batch_size=False,

    ## ------- PEFT------ ----- ##
    gradient_accumulation_steps=4,   # <-- enable
    #fp16=True,                       # <-- enable
    #fp16=False,                    # fp16 비활성화
    optim="adamw_bnb_8bit",

    ## ---- Learning Rate ----- ##
    warmup_steps=500,                # 워밍업을 위한 스텝 수
    weight_decay=0.01,               # 가중치 감소율

    ## ---- GPU & NODES ----- ##
    eval_strategy="epoch",

    ## - Logging & Checkpoint - ##
    output_dir='./results',          # 결과와 체크포인트를 저장할 디렉토리
    
    logging_steps=10,                # 로그를 기록할 스텝 간격
    #save_steps=1000,                 # 체크포인트를 저장할 스텝 간격
)


> Note!

fp16=True 등의 옵션은 가끔 라이브러리나 GPU 의 버젼에 따라 작동이 안될 수 있습니다. 일단 작은 데이터 사이즈로 테스트 해보고 잘 작동되면 적용하는 등의 시도를 해야 합니다.

### Trainer 설정

In [158]:
trainer = Trainer(
    model=model,
    args=training_args,

    # on Which Dataset
    train_dataset=train_dataset, # < train
    eval_dataset=test_dataset,   # < test
)

No label_names provided for model class `PeftModelForQuestionAnswering`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [159]:
result = trainer.train()
print_summary(result)

/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: in

Time: 1682.43
Samples/second: 12.48
GPU memory occupied: 30895 MB.


LoRA를 비롯한 다양한 테크닉을 쓰지 않았을 때는 14119 MB정도의 GPU 메모리가 필요한데, 2319MB 정도의 메모리만 쓰고 있는 것을 알 수 있습니다.

가지고 있는 메모리 용량등을 확인 한 후 batch size 나 gradient accumulation step등을 조정하시면서 성능을 최적화 하면 됩니다.

## Evaluation 수행


모델을 이용해 예측하는 것은 몇 가지 후처리를 필요로 합니다. 모델 자체는 답변의 시작과 끝 위치에 대한 로짓(logits)을 예측합니다. 만약 우리가 검증 데이터로더에서 배치를 가져온다면, 모델이 제공하는 출력은 다음과 같습니다

In [160]:
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [161]:
validation_features = dataset["validation"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=dataset["validation"].column_names
)

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [162]:
raw_predictions = trainer.predict(validation_features)

In [163]:
validation_features.set_format(
                type=validation_features.format["type"],
                columns=list(validation_features.features.keys())
)

위 Prediction 은 단순히 모델을 통해 예측한 값입니다.
몇 가지 예외사항등을 고려해서 다시 가공하는 루틴은 아래와 같습니다.

In [164]:
from tqdm.auto import tqdm
import collections
import numpy as np

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []

        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )

        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}

        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        predictions[example["id"]] = best_answer["text"]


    return predictions

In [165]:
final_predictions = postprocess_qa_predictions(
                        dataset["validation"],
                        validation_features,
                        raw_predictions.predictions
)

Post-processing 10570 example predictions split into 10784 features.


  0%|          | 0/10570 [00:00<?, ?it/s]

### 예측 결과 살펴보기

Validation 의 1번째 예제에 대해서 해봅니다.

In [166]:
import torch

for batch in trainer.get_eval_dataloader():
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()

odict_keys(['loss', 'start_logits', 'end_logits'])

In [167]:
max_answer_length = 30
n_best_size = 20

start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
offset_mapping = validation_features[0]["offset_mapping"]
# The first feature comes from the first example. For the more general case, we will need to be match the example_id to
# an example index
context = dataset["validation"][0]["context"]

# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
        # to part of the input_ids that are not in the context.
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
        # Don't consider answers with a length that is either < 0 or > max_answer_length.
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char: end_char]
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]
valid_answers

[{'score': 4.7578125, 'text': 'title'},
 {'score': 4.6992188,
  'text': 'game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver'},
 {'score': 4.5820312,
  'text': 'the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver'},
 {'score': 4.5625,
  'text': 'game to determine the champion of the National Football League (NFL)'},
 {'score': 4.5410156, 'text': 'game to determine the champion of the'},
 {'score': 4.515625,
  'text': '2015 season. The American Football Conference (AFC) champion Denver'},
 {'score': 4.515625,
  'text': 'of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver'},
 {'score': 4.506836, 'text': ') champion Denver'},
 {'score': 4.5,
  'text': ') for the 2015 season. The American Football Conference (AFC) champion Denver'},
 {'score': 4.473633,
  'text': ') champion Denv

In [168]:
dataset["validation"][0]["answers"]

{'text': ['Denver Broncos', 'Denver Broncos', 'Denver Broncos'],
 'answer_start': [177, 177, 177]}

정답이 잘 안나고 있는것을 확인할 수 있습니다.
이는 당연합니다. 현재 매우 작은 모델로 매우 작은 epoch만큼 훈련했기 때문입니다.
모델을 더 큰것을 쓰고, 훈련을 더 진행하게 되면 더 정확한 결과를 얻을 수 있습니다.

### Evaluation Metric준비하기


In [169]:
import evaluate  
metric = evaluate.load("squad")

In [170]:
validation_features

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'example_id'],
    num_rows: 10784
})

squad 의 metric을 사용하기 위해서는 정해진 형태대로 데이터를 만들어줘야 합니다.

In [171]:
formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in dataset["validation"]]
metric.compute(predictions=formatted_predictions, references=references)

{'exact_match': 0.9649952696310312, 'f1': 6.14831077576091}